In [5]:
pip install aif360

  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31279 sha256=f358743eb6761f906fc29a9dcef5bad7c7a321a3ee47e2d0d7e901a9fbe76ad3
  Stored in directory: c:\users\samah\appdata\local\pip\cache\wheels\01\ca\8b\b518dd2aef69635ad6fcab87069c9c52f355a2e9c5d4c02da9
Successfully built memory-profiler
  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1321 sha256=308012950fedca53bffb36552af6eb791c779f3ba4067adb6f491e182479ac02
  Stored in directory: c:\users\samah\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn



In [14]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [19]:
import numpy as np
np.set_printoptions(suppress = True)
import pandas as pd

import matplotlib.pyplot as plt

# Importing the Dataset
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

from aif360.metrics.utils import compute_boolean_conditioning_vector
#from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn

import pickle

In [44]:
from sklearn import svm

In [32]:
import sklearn.metrics

In [23]:
priv_group = [{'sex':1}]
unpriv_group = [{'sex':0}]

In [37]:
data_adult = load_preproc_data_adult(['sex'])

In [38]:
dset_raw_trn, dset_raw_tst = data_adult.split([0.7], shuffle=True)

In [39]:
dset_trn_pred = dset_raw_trn.copy(deepcopy=True)

In [41]:
def plot_prop_prtced_grp_lab(data, title="", figsize=(10,8)):
    feature_idx = np.where(np.array(data.feature_names) == data.protected_attribute_names[0])[0][0]
    n_p_fav = len(np.where(np.logical_and(data.features[:,feature_idx] == data.privileged_protected_attributes[0],data.labels == data.favorable_label) == True)[0])
    n_unp_fav = len(np.where(np.logical_and(data.features[:,feature_idx] == data.unprivileged_protected_attributes[0],data.labels == data.favorable_label) == True)[0])
    n_p_unfav = len(np.where(np.logical_and(data.features[:,feature_idx] == data.privileged_protected_attributes[0],data.labels == data.unfavorable_label) == True)[0])
    n_unp_unfav = len(np.where(np.logical_and(data.features[:,feature_idx] == data.unprivileged_protected_attributes[0],data.labels == data.unfavorable_label) == True)[0])
    plt.figure(figsize=figsize)
    xlocs = [i for i in range(4)]
    y = [n_p_fav/(n_p_fav+n_p_unfav), n_unp_fav/(n_unp_fav+n_unp_unfav), n_p_unfav/(n_p_fav+n_p_unfav), n_unp_unfav/(n_unp_fav+n_unp_unfav)]
    plt.bar(['priv grp,fav label','unpriv grp,fav label', 'priv grp,unfav label', 'unpriv grp,unfav label'], y)
    for i, v in enumerate(y):
        plt.text(xlocs[i] - 0.05, v + 0.01, str(round(v,2)))
    plt.title(title)
    plt.show()

In [42]:
dset_raw_trn_metrics = BinaryLabelDatasetMetric(dset_raw_trn,
                                      unprivileged_groups=unpriv_group,
                                     privileged_groups=priv_group)
print("Statistical Parity Difference:",dset_raw_trn_metrics.statistical_parity_difference())
print("Disparate Impact:",dset_raw_trn_metrics.disparate_impact())
print("Consistency:",dset_raw_trn_metrics.consistency()[0])

Statistical Parity Difference: -0.19312308652704469
Disparate Impact: 0.35996849011596344
Consistency: 0.7440463306911638


In [43]:
scaler = StandardScaler()
dset_trn = scaler.fit_transform(dset_raw_trn.features)
y_trn = dset_raw_trn.labels.ravel()
dset_trn = torch.from_numpy(dset_trn).float()
y_trn = torch.from_numpy(y_trn).float()
y_trn = y_trn.view(y_trn.shape[0], 1)

In [69]:
dat=dset_raw_trn.astype('pd.DataFrame')

AttributeError: 'AdultDataset' object has no attribute 'astype'

In [ ]:
svc=svm.SVC(kernel='linear',C=1).fit()